In [1]:
import pandas as pd
import os
import glob


In [2]:
data_folder = "./Data"
os.environ["DASHSCOPE_API_KEY"] = "sk-2ea9416b45e04af6b6aa72d3c2ade52f"

In [3]:
# import os
# import pandas as pd
# from openai import OpenAI
# import json

# # 设置文件路径
# data_folder = "Data"  # 假设数据文件夹名为 Data
# output_file = os.path.join(data_folder, "产品类别.txt")

# # 初始化OpenAI客户端，使用阿里云百炼平台
# client = OpenAI(
#     api_key=os.getenv("DASHSCOPE_API_KEY"),
#     base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
# )

# # 查找Data文件夹中以_processed.xlsx结尾的Excel文件
# excel_files = [f for f in os.listdir(data_folder) if f.endswith('_processed.xlsx')]
# if not excel_files:
#     print("在Data文件夹中未找到以_processed.xlsx结尾的Excel文件")
# else:
#     # 如果有多个文件，打印找到的所有文件
#     if len(excel_files) > 1:
#         print(f"找到以下{len(excel_files)}个_processed.xlsx文件:")
#         for i, file in enumerate(excel_files):
#             print(f"{i+1}. {file}")
    
#     # 使用第一个找到的Excel文件
#     excel_file = os.path.join(data_folder, excel_files[0])
#     print(f"正在处理文件: {excel_file}")
    
#     try:
#         # 读取Excel文件
#         df = pd.read_excel(excel_file)
        
#         # 检查必要的列是否存在
#         required_columns = ["subNodeLabel", "nodeLabelPath", "topNodeLabel"]
#         missing_columns = [col for col in required_columns if col not in df.columns]
#         if missing_columns:
#             print(f"Excel文件中缺少以下列: {', '.join(missing_columns)}")
#         else:
#             # 创建文件夹（如果不存在）
#             os.makedirs(data_folder, exist_ok=True)
            
#             # 打开输出文件
#             with open(output_file, 'w', encoding='utf-8') as f:
#                 # 遍历每个唯一的subNodeLabel
#                 for _, row in df.drop_duplicates(subset=["subNodeLabel"]).iterrows():
#                     sub_label = row["subNodeLabel"]
#                     node_path = row["nodeLabelPath"]
#                     top_label = row["topNodeLabel"]
                    
#                     # 构建提示词
#                     prompt = f"""请根据以下产品分类信息，确定一个最准确的产品类别名称：
#                     子类别: {sub_label}
#                     分类路径: {node_path}
#                     顶级类别: {top_label}
                    
#                     要求：
#                     1. 只输出一个最准确的产品类别名称
#                     2. 不要输出任何解释或说明
#                     3. 确保名称是从输入的内容中选择的
#                     """
                    
#                     # 使用流式输出调用Qwen API
#                     response = client.chat.completions.create(
#                         model="qwq-plus",  # 使用qwen-plus模型
#                         messages=[
#                             {"role": "system", "content": "你是一个专业的产品分类专家，能够准确识别和分类产品。"},
#                             {"role": "user", "content": prompt}
#                         ],
#                         temperature=0,
#                         stream=True
#                     )
                    
#                     # 收集流式响应
#                     content = ""
#                     for chunk in response:
#                         if chunk.choices[0].delta.content is not None:
#                             content += chunk.choices[0].delta.content
                    
#                     # 写入最终确定的产品类别
#                     f.write(f"{content.strip()}\n")
#                     print(f"已处理: {sub_label} -> {content.strip()}")
            
#             print(f"处理完成，结果已保存到{output_file}")
            
#     except Exception as e:
#         print(f"处理Excel文件时出错: {e}")

In [4]:
voc_files = glob.glob(os.path.join(data_folder, "*VOC数据.xlsx"))

if not voc_files:
    print("在Data文件夹中未找到以'VOC数据.xlsx'结尾的文件")
else:
    # 如果有多个文件，打印找到的所有文件
    if len(voc_files) > 1:
        print(f"找到以下{len(voc_files)}个VOC数据.xlsx文件:")
        for i, file in enumerate(voc_files):
            print(f"{i+1}. {file}")
    
    for voc_file in voc_files:
        print(f"正在处理VOC数据文件: {voc_file}")
        
        try:
            # 读取Excel文件的所有sheet
            excel = pd.ExcelFile(voc_file)
            sheet_names = excel.sheet_names
            
            # 只保留第一个sheet
            if sheet_names:
                first_sheet = sheet_names[0]
                df = pd.read_excel(voc_file, sheet_name=first_sheet)
                
                # 检查并重命名列
                if "星级" in df.columns:
                    df.rename(columns={"星级": "Rating"}, inplace=True)
                    print(f"已将'星级'列重命名为'Rating'")
                else:
                    print(f"未找到'星级'列")
                
                if "内容" in df.columns:
                    df.rename(columns={"内容": "content"}, inplace=True)
                    print(f"已将'内容'列重命名为'content'")
                else:
                    print(f"未找到'内容'列")
                
                # 保存修改后的文件
                with pd.ExcelWriter(voc_file, engine='openpyxl') as writer:
                    df.to_excel(writer, sheet_name=first_sheet, index=False)
                    print(f"已成功保存修改后的文件，只保留第一个sheet: {first_sheet}")
            else:
                print(f"Excel文件中没有sheet")
                
        except Exception as e:
            print(f"处理VOC数据文件时出错: {e}")

正在处理VOC数据文件: ./Data/大汇镭射膜-VOC数据.xlsx
已将'星级'列重命名为'Rating'
已将'内容'列重命名为'content'
已成功保存修改后的文件，只保留第一个sheet: Worksheet


已成功保存修改后的文件，只保留第一个sheet: Worksheet
